# Compute some potentials from QE

In [1]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput, QEOutput

In [2]:
qe_options = {
    '&control': {
        'prefix' : "'tmp'",
        'outdir' : "'./'",
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 2,
        'ntyp': 1,
        'occupations': "'smearing'",
        'input_dft' : "'PBE'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['2 2 2 0 0 0'],
}

In [3]:
from ase.build import bulk
atoms = bulk('Al', 'fcc', a = 4.05, cubic=True)
atoms.positions[0,0] = 0.2

In [4]:
driver=Driver(qe_options=qe_options, logfile=True, atoms=atoms)

In [5]:
driver.electrons()

-551.5572382538037

In [6]:
# driver.qepy.control_flags.set_iverbosity(1)
# forces_total = driver.get_forces()
# stress_total = driver.get_stress()

In [7]:
output = driver.get_output()

## Compute Forces and Stress

> **NOTE** `Turn the following on to print all the information`

In [8]:
# driver.qepy.control_flags.set_iverbosity(1)

In [9]:
forces_total = driver.get_forces()
output_forces = driver.get_output()
output_forces

['\n',
 '     Forces acting on atoms (cartesian axes, Ry/au):\n',
 '\n',
 '     atom    1 type  1   force =    -0.03330810    0.00000000    0.00000000\n',
 '     atom    2 type  1   force =    -0.00527465    0.00000000    0.00000000\n',
 '     atom    3 type  1   force =     0.01929137    0.00000000    0.00000000\n',
 '     atom    4 type  1   force =     0.01929137    0.00000000    0.00000000\n',
 '\n',
 '     Total force =     0.043377     Total SCF correction =     0.000092\n']

In [10]:
stress_total = driver.get_stress()
output_stress = driver.get_output()
output_stress

['\n',
 '\n',
 '     Computing stress (Cartesian axis) and pressure\n',
 '\n',
 '          total   stress  (Ry/bohr**3)                   (kbar)     P=    -2825.01\n',
 '  -0.01935794   0.00000000   0.00000000        -2847.65        0.00        0.00\n',
 '   0.00000000  -0.01912710   0.00000000            0.00    -2813.69        0.00\n',
 '   0.00000000   0.00000000  -0.01912710            0.00        0.00    -2813.69\n',
 '\n']

In [11]:
forces = QEOutput.get_forces_all(output_forces, nat=len(atoms))
# forces = QEOutput.get_forces_all(output, nat=len(atoms))

In [12]:
forces

{'total': array([[-0.0333081 ,  0.        ,  0.        ],
        [-0.00527465,  0.        ,  0.        ],
        [ 0.01929137,  0.        ,  0.        ],
        [ 0.01929137,  0.        ,  0.        ]])}

In [13]:
if len(forces)>2:
    error=0.0
    for k, v in forces.items():
        if k=='total' :
            error -= v
        else:
            error += v
    print(error)

In [14]:
stress = QEOutput.get_stress_all(output_stress)
# stress = QEOutput.get_stress_all(output)

In [15]:
stress

{'total': array([[-2847.65,     0.  ,     0.  ],
        [    0.  , -2813.69,     0.  ],
        [    0.  ,     0.  , -2813.69]])}

In [16]:
if len(stress) > 1:
    error=0.0
    for k, v in stress.items():
        if k=='total' :
            error -= v
        else:
            error += v
    print(error)